In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
df = pd.read_json("/kaggle/input/sarcasm-detection-through-nlp/Sarcasm_Headlines_Dataset.json", lines = True)
df.head()

In [ ]:
df.to_csv('sarcasm_csv', index = False)
df = pd.read_csv("/kaggle/working/sarcasm_csv")
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
y = df.is_sarcastic
x = df.drop('is_sarcastic', axis = 1)

In [ ]:
print(x.shape)
print(y.shape)

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout, BatchNormalization
from keras.regularizers import l2
from keras.optimizers import RMSprop

In [ ]:
voca_size = 10000

In [ ]:
text = x.copy()

In [ ]:
text['headline'][1]

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(len(text)):
    review = re.sub('[^a-zA-Z]', ' ', text['headline'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

In [ ]:
onehot_repr=[one_hot(words,voca_size)for words in corpus]
onehot_repr

In [ ]:
print(corpus[1])
print(onehot_repr[1])

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

In [ ]:
print(embedded_docs[1])

In [ ]:
X_train_reshaped = np.expand_dims(X_train, axis=-1)
X_test_reshaped = np.expand_dims(X_test, axis=-1)

In [ ]:
embedding_vector_features = 50  
model = Sequential()
model.add(Embedding(voca_size, embedding_vector_features))
model.add(Bidirectional(LSTM(128, return_sequences=True)))  
model.add(Dropout(0.2))  
model.add(Bidirectional(LSTM(64, return_sequences=True)))  
model.add(Dropout(0.2))  
model.add(Bidirectional(LSTM(32)))  
model.add(Dropout(0.2))  
model.add(Dense(1, activation='sigmoid'))

optimizer_rmsprop = RMSprop(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer_rmsprop, metrics=['accuracy'])

In [ ]:
len(embedded_docs),y.shape

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=64)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
y_pred=np.where(y_pred > 0.6, 1,0)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))